In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [2]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')

In [3]:
def label_encoder(c):
    lc = LabelEncoder()
def age_range(age):
    if age >= 0 and age <= 5:
        return "baby"
    elif age >= 6 and age <= 16:
        return "children"
    elif age >= 17 and age <= 29:
        return "young_adult"
    elif age >= 30 and age <=49:
        return "middle_aged"
    else:
        return "senior"
    return lc.fit_transform(c)
def preprocess(df):
    age_map = df[['Age', 'Pclass']].dropna().groupby('Pclass').mean().to_dict()
    df.Age = df.Age.fillna(df.Pclass.map(age_map['Age']))
    df.Cabin = df.Cabin.fillna('X').map(lambda x: x[0].strip())
    df.Ticket = df.Ticket.fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')
    df.Fare = df.Fare.fillna(df.Fare.mean())
    df.Fare = df.Fare.map(lambda i: np.log(i) if i > 0 else 0)
    df.Embarked = df.Embarked.fillna('X')
    df.Name = df.Name.map(lambda x: x.split(',')[0])
    df["age_range"] = df["Age"].apply(age_range)
    df["Family"] = df["SibSp"] + df["Parch"] +1   
    df["Alone"] =  df["Family"].map(lambda x : "Yes" if x == 1 else "No")
    label_cols = ['Name', 'Ticket',"age_range","Alone"]
    onehot_cols = ['Pclass', 'Sex', 'Cabin', 'Embarked']
    numerical_cols = ['SibSp', 'Parch', 'Fare','Survived',"Family"]  
    onehot_encoded_df = pd.get_dummies(df[onehot_cols])
    label_encoded_df = df[label_cols].apply(label_encoder)
    numerical_df = df[numerical_cols]
    return pd.concat([numerical_df,onehot_encoded_df], axis=1)

def test_preprocess(df):
    age_map = df[['Age', 'Pclass']].dropna().groupby('Pclass').mean().to_dict()
    df.Age = df.Age.fillna(df.Pclass.map(age_map['Age']))
    df.Cabin = df.Cabin.fillna('X').map(lambda x: x[0].strip())
    df.Ticket = df.Ticket.fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')
    df.Fare = df.Fare.fillna(df.Fare.mean())
    df.Fare = df.Fare.map(lambda i: np.log(i) if i > 0 else 0)
    df.Embarked = df.Embarked.fillna('X')
    df.Name = df.Name.map(lambda x: x.split(',')[0])
    df["age_range"] = df["Age"].apply(age_range)
    df["Family"] = df["SibSp"] + df["Parch"] +1   
    df["Alone"] =  df["Family"].map(lambda x : "Yes" if x == 1 else "No")
    label_cols = ['Name', 'Ticket',"age_range","Alone"]
    onehot_cols = ['Pclass', 'Sex', 'Cabin', 'Embarked']
    numerical_cols = ['SibSp', 'Parch', 'Fare',"Family"]  
    onehot_encoded_df = pd.get_dummies(df[onehot_cols])
    label_encoded_df = df[label_cols].apply(label_encoder)
    numerical_df = df[numerical_cols]
    return pd.concat([numerical_df,onehot_encoded_df], axis=1)

In [4]:
persudo = pd.read_csv("../input/apr-play-ground-persudo/persudosubmit.csv")

In [5]:
train = preprocess(df = train_df)
test = test_preprocess(df=test_df)
persudo_test = test.copy()
persudo_test['Survived'] = persudo["Survived"]
full_df = train.append(persudo_test)
y = full_df.pop("Survived")
x = full_df.values


In [6]:
 params = {'criterion': 'entropy',
  'splitter': 'best',
  'max_depth': 1559,
  'min_samples_split': 742,
  'min_samples_leaf': 111,
  'max_leaf_nodes': 1512}

In [7]:
test_acc = DecisionTreeClassifier(**params)
sk_fold = StratifiedKFold(5)
print(cross_val_score(test_acc,x,y, n_jobs=-1, cv=sk_fold,scoring='accuracy').mean())
print(cross_val_score(test_acc,x,y, n_jobs=-1, cv=sk_fold,scoring='f1').mean())

0.8750450000000001
0.8357320405418183


In [8]:
clf = DecisionTreeClassifier(**params)
clf.fit(x,y)

DecisionTreeClassifier(criterion='entropy', max_depth=1559, max_leaf_nodes=1512,
                       min_samples_leaf=111, min_samples_split=742)

In [9]:
preds = clf.predict(test)

In [10]:
submit = pd.DataFrame({"PassengerId":test_df["PassengerId"],"Survived":preds})
submit.head(10)

,PassengerId,Survived
0,100000,0
1,100001,1
2,100002,1
3,100003,0
4,100004,1
5,100005,0
6,100006,1
7,100007,0
8,100008,0
9,100009,0


In [11]:
submit.to_csv("submission.csv",index = False)